In [8]:
print("Hello World!")

Hello World!


# CA
## Subject:
### Student: Vitalis Smirnovs
### Student ID:

#### Document Outline:
1. Introduction;
2. Data and Methods;
3. Regression Analysis;
4. Machine Learning;
5. Discussion of Results and Conclusion.

1. Introduction

1.1. Crop Yeld In Ireland <br>
1.2. Data: Source and Description <br>
1.3. Methods <br>

In [9]:
# import libraries that are needed for this project
import numpy as np 
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as pltpy
import seaborn as sns
import scipy.stats as stats
import termcolor
from termcolor import colored
#from astroML.plotting import setup_text_plots
#import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score
%matplotlib inline
from scipy.optimize import curve_fit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pickle 

In [10]:
# https://forums.fast.ai/t/jupyter-notebook-enhancements-tips-and-tricks/17064/5
# these are the functions to help to make output look nicer and easier to read
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))                                           # for printing in bold

def printmd(string, color=None):                                               
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)  # printing in colour
    display(Markdown(colorstr))

In [11]:
# data source: https://data.gov.ie/dataset/aqa03-crop-yield-1985-2007/resource/2bebaae8-f9a0-4080-9e67-a1f2e130a943
df = pd.read_csv('dataCA2_v2.csv') # read in the data into a dataframe
df                                      # display dataframe

,Year,Code,Crop,CropArea,Yield_ha,Yield
0,1985,1,"Total wheat, oats and barley",386.2,5.1,1986.0
1,1985,11,Total wheat,77.7,6.4,495.0
2,1985,111,Winter wheat,60.3,6.5,395.0
3,1985,112,Spring wheat,17.4,5.8,100.0
4,1985,12,Total oats,24.9,5.1,126.0
...,...,...,...,...,...,...
386,2007,4,Potatoes,11.7,34.0,399.0
387,2007,5,Turnips,NaN,NaN,NaN
388,2007,61,Sugar beet,NaN,NaN,NaN
389,2007,62,Fodder beet,NaN,NaN,NaN


In [7]:
# have a look at the data, to see what it looks like
printmd("**Table 1: Data info**", color='red')      
print("----------------------------------")
print(df.info())                                    # print data info on screen

printmd("**Table 2: Data head**", color='red')     
print("----------------------------------")
print(df.head())                                    # view top of the data set

printmd("**Table 3: Data tail**", color='red')
print("----------------------------------")
print(df.tail())                                    #view bottom of the data set#
printmd("**Table 4: Data Summary Stats**", color='red')                 # print table numbers and headings
printmd("----------------------------------------------", color='red')  # separate output by a line
print(df.describe())     

<span style='color:red'>**Table 1: Data info**</span>

----------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      391 non-null    int64  
 1   Code      391 non-null    int64  
 2   Crop      391 non-null    object 
 3   CropArea  368 non-null    float64
 4   Yield_ha  368 non-null    float64
 5   Yield     368 non-null    float64
dtypes: float64(3), int64(2), object(1)
memory usage: 18.5+ KB
None


<span style='color:red'>**Table 2: Data head**</span>

----------------------------------
   Year  Code                          Crop  CropArea  Yield_ha   Yield
0  1985     1  Total wheat, oats and barley     386.2       5.1  1986.0
1  1985    11                   Total wheat      77.7       6.4   495.0
2  1985   111                  Winter wheat      60.3       6.5   395.0
3  1985   112                  Spring wheat      17.4       5.8   100.0
4  1985    12                    Total oats      24.9       5.1   126.0


<span style='color:red'>**Table 3: Data tail**</span>

----------------------------------
     Year  Code                    Crop  CropArea  Yield_ha  Yield
386  2007     4                Potatoes      11.7      34.0  399.0
387  2007     5                 Turnips       NaN       NaN    NaN
388  2007    61              Sugar beet       NaN       NaN    NaN
389  2007    62             Fodder beet       NaN       NaN    NaN
390  2007     7  Kale and field cabbage       NaN       NaN    NaN


<span style='color:red'>**Table 4: Data Summary Stats**</span>

<span style='color:red'>----------------------------------------------</span>

              Year        Code    CropArea    Yield_ha        Yield
count   391.000000  391.000000  368.000000  368.000000   368.000000
mean   1996.000000   53.529412   60.465761   16.276902   536.256522
std       6.641748   53.407562   85.212043   18.614356   578.540756
min    1985.000000    1.000000    0.000000    0.000000     0.000000
25%    1990.000000    5.000000    8.350000    5.900000    89.525000
50%    1996.000000   13.000000   20.750000    7.200000   305.500000
75%    2002.000000  112.000000   68.600000   12.950000   795.525000
max    2007.000000  132.000000  386.200000   73.700000  2500.900000


In [14]:
# climate data
# https://www.met.ie/climate/available-data/long-term-data-sets/
df = pd.read_csv('climateData1.csv') # read in the data into a dataframe
df                                      # display dataframe


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1850,113.5,86.4,38.6,138.3,59.7,54.5,103.4,76.3,70.5,62.7,113.1,104.0
1,1851,195.0,55.2,79.9,54.7,45.0,101.5,90.3,96.6,49.4,113.4,45.4,68.8
2,1852,161.4,92.4,44.6,44.4,70.2,179.4,68.6,110.1,57.3,88.7,223.2,208.2
3,1853,144.7,43.0,83.5,75.3,33.2,77.5,100.5,82.9,60.2,185.1,106.0,48.4
4,1854,138.6,36.0,37.3,18.8,100.0,110.4,102.6,60.5,47.8,77.8,74.6,94.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,2006,57.6,54.6,120.3,47.3,126.2,29.8,55.9,70.8,162.6,145.6,142.3,170.5
157,2007,107.9,107.8,80.3,26.9,65.9,141.3,133.0,100.4,52.8,52.2,75.0,123.1
158,2008,174.3,58.0,113.5,45.1,42.3,101.9,123.8,175.9,108.2,152.4,80.4,73.7
159,2009,152.9,36.5,48.7,120.9,90.7,73.2,174.7,148.4,49.4,129.9,244.9,105.4


In [27]:
# clculate average yearly temperature
df['averageT'] = (df['Jan'] +  df['Feb'] +  df['Mar'] 
                  +  df['May'] +  df['Jun'] +  df['Jul'] 
                  +  df['Aug'] +  df['Sep'] +  df['Oct'] 
                  +  df['Nov'] +  df['Apr'] +  df['Dec']) / 12

In [16]:
df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,averageT
0,1850,113.5,86.4,38.6,138.3,59.7,54.5,103.4,76.3,70.5,62.7,113.1,104.0,85.083333
1,1851,195.0,55.2,79.9,54.7,45.0,101.5,90.3,96.6,49.4,113.4,45.4,68.8,82.933333
2,1852,161.4,92.4,44.6,44.4,70.2,179.4,68.6,110.1,57.3,88.7,223.2,208.2,112.375000
3,1853,144.7,43.0,83.5,75.3,33.2,77.5,100.5,82.9,60.2,185.1,106.0,48.4,86.691667
4,1854,138.6,36.0,37.3,18.8,100.0,110.4,102.6,60.5,47.8,77.8,74.6,94.9,74.941667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,2006,57.6,54.6,120.3,47.3,126.2,29.8,55.9,70.8,162.6,145.6,142.3,170.5,98.625000
157,2007,107.9,107.8,80.3,26.9,65.9,141.3,133.0,100.4,52.8,52.2,75.0,123.1,88.883333
158,2008,174.3,58.0,113.5,45.1,42.3,101.9,123.8,175.9,108.2,152.4,80.4,73.7,104.125000
159,2009,152.9,36.5,48.7,120.9,90.7,73.2,174.7,148.4,49.4,129.9,244.9,105.4,114.633333


In [26]:
# keep only years after 1985 & before 2007
df = df[(df.Year >= 1985)]
df = df[(df.Year <= 2007)]

In [25]:
df.describe()


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,averageT
count,23.00000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,1996.00000,111.556522,88.991304,87.686957,78.113043,72.026087,77.595652,75.508696,93.834783,83.386957,122.700000,109.578261,117.534783,93.209420
std,6.78233,41.761790,45.292031,29.239797,27.496550,36.383366,32.189954,25.309231,45.646255,39.949563,47.960476,38.987492,41.664980,9.009915
min,1985.00000,35.200000,6.500000,35.700000,26.900000,7.700000,29.800000,23.800000,13.000000,7.500000,52.200000,54.200000,52.000000,77.058333
25%,1990.50000,76.100000,58.100000,65.200000,59.700000,43.850000,53.600000,58.900000,67.900000,58.550000,67.400000,77.050000,88.050000,87.629167
50%,1996.00000,123.400000,79.900000,90.900000,79.900000,67.900000,79.700000,73.900000,83.700000,80.200000,144.500000,102.600000,119.100000,93.866667
75%,2001.50000,140.300000,110.650000,108.600000,100.000000,86.850000,98.700000,87.400000,115.650000,99.850000,160.750000,133.900000,147.100000,98.237500
max,2007.00000,194.600000,192.500000,134.300000,124.000000,139.500000,141.300000,133.000000,191.500000,173.600000,199.100000,187.300000,192.800000,116.108333
